In [6]:
import json
import os.path
import time
import sqlite3
import torch
from torchvision import transforms
from tqdm import tqdm
from dataset.FastPatchExtractor import FastImageLoader
from regularisers import PatchNrRegulariser

In [5]:
params_path = 'results/prior_variance_experiment/custom_material_prior/version_1/params.json'
with open(params_path, 'r') as fp:
        params_dict = json.load(fp)
value_db_path = os.path.join(params_dict['result_dir'], 'values.db')

FileNotFoundError: [Errno 2] No such file or directory: 'results/prior_variance_experiment/custom_material_prior/version_1/params.json'

In [ ]:
connection = sqlite3.connect(value_db_path)
cursor = connection.cursor()

In [ ]:
normalization = transforms.Compose([transforms.Normalize([0, ], [255., ])])
image_loader = FastImageLoader(image_path, device=DEVICE, transform=normalization)
ground_truth = image_loader[0].to(DEVICE)

In [ ]:
steps = [i for i in range(start, end, step_size)]

In [ ]:
res = []
times = []
for current_step in range(start, end, step_size):
    custom_regulariser = PatchNrRegulariser(model, p_size=patch_size, sample_number=current_step, padding=True, padding_size=16, device=DEVICE)
    values = []
    start_time = time.time()
    for i in tqdm(range(250)):
        custom_val = custom_regulariser.evaluate(ground_truth)
        values.append(custom_val.item())
    end_time = time.time()
    values = torch.tensor(values)
    val_min = torch.min(values)
    val_mean = torch.mean(values)
    val_max = torch.max(values)
    res.append((val_min, val_mean, val_max))
    times.append(end_time-start_time)